In [ ]:
import os

token = os.environ["AUTHTOKEN"] # your authentication token
url_endpoint = os.environ["URL"] # api endpoint
method = os.environ["METHOD"] # rpc method call
item_id = os.environ["ITEMID"] # id of item 

In [ ]:
from jsonrpcclient.clients.http_client import HTTPClient
from jsonrpcclient.requests import Request

client = HTTPClient(url_endpoint)
headers = {
    "content-type": "application/json",
    "authorization": "Bearer " +  token
}

client.session.headers.update(headers)

rpc_text = Request(
    method,
    query = { "id": item_id},
    window = {
        "from": "2017-01-20T00:00:00Z",
        "to": "2021-12-21T00:00:00Z"
    }
)

res = client.send(rpc_text)
data = res.data.result["sections"][0]

In [ ]:
import numpy as np
import pandas as pd

ts = np.array(data)
time, value = pd.to_datetime(ts[:,0]), pd.to_numeric(ts[:,1])
series = pd.Series(value, index=time)
series.plot()